In [2]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from statsmodels.tools import add_constant
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import statsmodels.api as sm
sns.set(style="ticks")
import random
%matplotlib inline

In [3]:
pm = pd.read_csv("fin_pmweather.csv")

In [7]:
#takes out features columns
x = pm.iloc[:, 3:]


#scale x-variables
scaled = preprocessing.minmax_scale(x)
x_scaled = pd.DataFrame(scaled)

#reassign correct column names
cols = list(x.columns)
x_scaled.columns = cols

#takes of what we want to predict
target = pm.iloc[:, 2]
target = 100*target

x_train, x_test, y_train, y_test = train_test_split(x_scaled, target, test_size=0.30, random_state=0)

#add random noise
noise = np.abs(np.random.normal(0, 1, len(x_train)))

In [23]:

#C(0.91, (0.70, 1.12))
#length scale is sd of y_train, constant term captures the entire range of y_train plus some more so that amplitude of kernel
# will capture most/all of y-values
#C(0.05, (3.05e-07, 0.12))
#C(1, (0.001, 10))*
kernel =(1.32**2)* RBF(length_scale = 0.00911)
gp = GaussianProcessRegressor(kernel=kernel, alpha = 0.0001, n_restarts_optimizer=9)
gp.fit(x_train, y_train)

y_train_pred, sigma_train = gp.predict(x_train, return_std=True)
y_test_pred, sigma_test = gp.predict(x_test, return_std=True)

#get R^2
r2 = gp.score(x_train, y_train)
r2_t = gp.score(x_test, y_test)

#get MSE measurements
MSE_test = np.mean((y_test_pred - y_test)**2)
MSE_train = np.mean((y_train_pred - y_train)**2)

#get log likelihood
t=gp.log_marginal_likelihood()

#calculate AIC
AIC = 2*len(x_test.columns) - 2*np.log(-t)
print(r2, r2_t)
gp.kernel_

0.621389731835 -2.18336385813


1.32**2 * RBF(length_scale=0.00911)